In [92]:
object Ax{
    def apply[P](a: P): P = a
}

type ⟶[A, B] = A => B

object ⟶{
    def I[A, B](f: A => B): A => B =
        f
    
    def E[A, B](f: A => B)(a: A): B = 
        f(a)
}

type ⊥ = Nothing 

object ⊥{
    def E[A](a: ⊥): A = a
} 

type ∨[A, B] = Either[A, B]

object ∨{
    def IL[A, B](a: A): A ∨ B = Left(a)

    def IR[A, B](b: B): A ∨ B = Right(b)
    
    def E[A, B, C](f: A => C, g: B => C)(or: A ∨ B): C = 
        or.fold(f, g)
}

type ∧[A, B] = (A, B)

object ∧{
    def I[A, B](a: A, b: B): A ∧ B = (a, b)
    
    def EL[A, B](and: A ∧ B): A = and._1
    
    def ER[A, B](and: A ∧ B): B = and._2
}

defined object Ax
defined type ⟶
defined object ⟶
defined type ⊥
defined object ⊥
defined type ∨
defined object ∨
defined type ∧
defined object ∧

In [70]:
type ¬[A] = A ⟶ ⊥
type ⟷[A, B] = (A ⟶ B) ∧ (B ⟶ A)
type ⟵[A, B] = B ⟶ A

defined type ¬
defined type ⟷
defined type ⟵

In [76]:
type Absurd = ⊥
type Or[A, B] = ∨[A, B] 
type And[A, B] = ∧[A, B]
type Not[A] = ¬[A]
type <=>[A, B] = (A => B, B => A)
val Implies = ⟶
val Or = ∨ 
val And = ∧

defined type Absurd
defined type Or
defined type And
defined type Not
defined type <=>
Implies: ⟶.type = ammonite.$sess.cmd55$Helper$$u27F6$@75f0a435
Or: ∨.type = ammonite.$sess.cmd55$Helper$$u2228$@1627b976
And: ∧.type = ammonite.$sess.cmd55$Helper$$u2227$@6e3ef3a1

$\vdash X \wedge (X \rightarrow Y) \rightarrow Y$

In [82]:
def proof[X, Y]: (X => Y, X) => Y = 
    //(x, f) => f(x)
    _(_)

defined function proof

In [83]:
def proof[X, Y]: ((X, X => Y)) => Y = 
    (a: (X, X => Y)) => 
        (a._2)(a._1)

defined function proof

In [84]:
def proof[X, Y]: X ∧ (X ⟶ Y) ⟶ Y = 
    ⟶.I{ a: X ∧ (X ⟶ Y) => 
        ⟶.E(∧.ER(a))(∧.EL(a))
    }

defined function proof

$X*(Y+Z) \cong X*Y + X*Z$

$\vdash X \wedge (Y \vee Z) \leftrightarrow X \wedge Y \vee X \wedge Z$

In [85]:
def proof[P, Q, R]: (P ∧ (Q ∨ R)) ⟷ ((P ∧ Q) ∨ (P ∧ R)) = 
    ∧.I(
        ⟶.I{ a: P ∧ (Q ∨ R) => 
            ∨.E((q: Q) => ∨.IL(∧.I(∧.EL(a), q)),
                (r: R) => ∨.IR(∧.I(∧.EL(a), r)))(∧.ER(a))
        },
        ⟶.I{ 
            ∨.E((a: P ∧ Q) => ∧.I(∧.EL(a), ∨.IL(∧.ER(a))),
                (a: P ∧ R) => ∧.I(∧.EL(a), ∨.IR(∧.ER(a))))
        }
    )

defined function proof

In [86]:
def proof[P, Q, R]: (P, Either[Q, R]) <=> Either[(P, Q), (P, R)] = 
    ({ case (p, Left(q)) => Left((p,q))
        case (p, Right(r)) => Right((p,r)) },
    { case Left((p,q)) => (p, Left(q))
        case Right((p,r)) => (p, Right(r)) })

defined function proof

$X^{Y+Z} \cong X^Y * X^Z$

$\vdash (Y \vee Z \rightarrow X) \leftrightarrow (Y \rightarrow X) \wedge (Z \rightarrow X)$

In [87]:
def proof[X, Y, Z]: (Y ∨ Z => X) <=> (Y => X ∧ Z => X) = 
    (??? : (Y ∨ Z => X) => (Y => X ∧ Z => X), 
     ??? : (Y ∨ Z => X) <= (Y => X ∧ Z => X))

defined function proof

In [88]:
def proof[X, Y, Z]: (Either[Y, Z] => X) <=> (Y => X, Z => X) = 
    (f => (y => f(Left(y)), z => f(Right(z))) , 
     { case (f, g) => 
         { case Left(y) => f(y)
           case Right(z) => g(z)
         }
     })

defined function proof

$\vdash P \vee Q \rightarrow \neg P \rightarrow Q$ 

In [89]:
def proof[P, Q]: Either[P, Q] => (P => Nothing) => Q = 
    (e: Either[P, Q]) => (f: P => Nothing) => 
        //e.fold[B](f, identity)
        e match {
            case Left(a: P) => f(a): Q
            case Right(b: Q) => b : Q
        }

defined function proof

In [93]:
def proof[P, Q]: P ∨ Q ⟶ (¬[P] ⟶ Q) = 
    ⟶.I{ or: P ∨ Q => 
        ⟶.I{ n: ¬[P] => 
            ∨.E((p: P) => ⊥.E[Q](⟶.E(n)(p)), 
                Ax[Q])(or)
        }
    }

defined function proof

In [29]:

def proof8[A, B]: Or[A, B] => Or[B, A] = 
    Or.elim[A, B, Or[B, A]](Or.applyR[B, A], Or.applyL[B, A])

// Not and implies

def proof3[A, B]: (A => Not[B]) => (A => B) => Not[A] = 
    i => (i2: A => B) => (a: A) => i(a)(i2(a))

def proof2[A, B]: Not[A] => A => B = 
    n => Not.elim[A, B](n)

def proof4[A, B]: Not[A => B] => Not[B] = 
    n => (b: B) => n((_: A) => b)

// Not and and

def proof5[A, B]: Not[A] => Not[And[A, B]] = 
//    (and: And[A, B]) => n(And.elimL(and))
    n => n compose And.elimL

def proof7[A, B]: Not[And[A, B]] => A => Not[B] = 
    n => (a: A) => (b: B) => n(And(a, b))

def proof11[A]: Not[A And Not[A]] = 
    a => And.elimR(a)(And.elimL(a))

// Not and or

def proof9[A, B]: Not[A Or B] => Not[A] And Not[B] = 
    n => And((a: A) => n(Or.applyL(a)), (b: B) => n(Or.applyR(b)))

def proof10[A, B]: (Not[A] And Not[B]) => Not[A Or B] = 
    and => Or.elim[A, B, Absurd](And.elimL(and), And.elimR(and))



defined function proof8
defined function proof3
defined function proof2
defined function proof4
defined function proof41
defined function proof5
defined function proof7
defined function proof11
defined function proof9
defined function proof10

In [13]:
trait ClassicalLogic{
    trait DN{
        def apply[A](nn: Not[Not[A]]): A
    }
    
    val dn: DN
    
    def proof1[A]: (Not[A] => A) => A =
        f => dn((n: Not[A]) => proof11(And(f(n),n)))
}


defined trait ClassicalLogic

All P are S
All S are R
All P are R

In [15]:
def proof[P, S, R]: ((P => S, S => R)) => P => R = 
    t => (p: P) => t._2(t._1(p))

defined function proof

AG. $\{ r \vee p \rightarrow \neg p, p \rightarrow q, \neg q \vee p\} \vdash \neg p$

In [113]:
def proof[P, Q, R](
    p1: R Or P => Not[P], 
    p2: P => Q, 
    p3: Not[Q] Or P): Not[P] = 
    (p: P) => p1(Right(p))(p) : Absurd

defined function proof

$ \vdash (p \rightarrow (q \rightarrow r)) \rightarrow (p \rightarrow q) \rightarrow p \rightarrow r$

In [38]:
def term[P, Q, R]: (P => (Q => R)) => (P => Q) => (P => R) = 
    (p1: P => Q => R) => 
        (p2: P => Q) => 
            (p: P) => p1(p : P)(p2(p) : Q) : R

defined function term

In [94]:
def proof[P, Q, R]: (P ⟶ (Q ⟶ R)) ⟶ ((P ⟶ Q) ⟶ (P ⟶ R)) = 
    ⟶.I{ p1: P ⟶ (Q ⟶ R) => 
        ⟶.I{ p2: P ⟶ Q => 
            ⟶.I{ p: P =>
                ⟶.E(⟶.E(p1)(p))(⟶.E(p2)(p))
            }
        }
    }

defined function proof

$ \vdash p \rightarrow q \rightarrow p$

In [39]:
def proof[P, Q, R]: P => Q => P = 
    (p: P) => 
        (q: Q) => p : P

defined function proof

In [96]:
def proof[P, Q, R]: P ⟶ (Q ⟶ P) = 
    ⟶.I{ p: P => 
        ⟶.I{ q: Q => 
            p : P
        }
    }

defined function proof

In [98]:
def proof[A, B, C]: (A => (B => C)) => (A Either B => C) = 
    (p: A => B => C) => 
        (e: Either[A, B]) => 
            e.fold(
                (a: A) => ??? : C, 
                (b: B) => ??? : C)

defined function proof

$\vdash \neg \neg \neg p \leftrightarrow \neg p$

In [103]:
def proof[P]: Not[Not[Not[P]]] => Not[P] = 
    (nnnp: ((P => Absurd) => Absurd) => Absurd) => 
        (p: P) => 
            nnnp( (np : P => Absurd) => np(p) : Absurd): Absurd

defined function proof

In [104]:
def proof[P]: Not[Not[Not[P]]] => Not[P] = 
    nnnp => p => nnnp(_(p))

defined function proof

Ejercicio AG.

In [107]:
def proof[P, Q, R, S, T](
    p1: P => Not[R] Or S,
    p2: Not[R] => S And Not[T],
    p3: P => T,
    p4: P => Not[S]): Not[P] = 
    (p: P) => 
        p1(p) match {
            case nr: Not[R] => p4(p)(p2(nr)._1) : Absurd
            case s: S => p4(p)(s) : Absurd
        }

defined function proof

In [112]:
def proof[P, Q, R, S, T](
    p1: P ⟶ (¬[R] ∨ S),
    p2: ¬[R] ⟶ (S ∧ ¬[T]),
    p3: P ⟶ T,
    p4: P ⟶ ¬[S]): ¬[P] = 
    ⟶.I((p: P) => 
        ∨.E(
            (nr: Not[R]) => 
                ⟶.E(⟶.E(p4)(p))(∧.EL(⟶.E(p2)(nr))) : Absurd,
            (s: S) => 
                ⟶.E(⟶.E(p4)(p))(s) : Absurd
        )(⟶.E(p1)(p)))

defined function proof